# To Do
1. Quality checks

We're using os and glob to do bash script stuff.

In [4]:
import os
import glob
import pandas as pd
import numpy as np

#stores the working directory as a string
path = os.getcwd()

#ref folder name, ref genomes go here
ref_folder = 'references'

#data folder name, samples and runs go here
data = 'data'

#csv output folder name
csv = 'Ex'


# Get List of SRR#s to Automate Downloading Data

Use Georgia's metadata table to download sample runs. 

Metadata Table -> dataframe -> list of SRR#s

List of SRR#s -> SRA Toolkit -> fastq files -> salmon -> quant files

In [5]:
#read in SRR#s from metadata table provided by Georgia
sra_run_table = pd.read_csv('SraRunTable.csv', sep=",", header=0, index_col=0)
print(sra_run_table['Run'][3:5])

4    SRR8239646
5    SRR6483189
Name: Run, dtype: object


# Building Index
Use salmon to build index. Download genomes from ensemble and store in refences folder. Will align to references in parallel. This is written to align to as many reference genomes/transcriptomes as are put in the references folder. 

In [6]:
#grabs stuff and returns a list of them-used throughout
def grab_ref(ref_folder1):
    files=glob.glob(ref_folder1+'/*')
    #print(files)
    return(files)

#builds Salmon index from given file names, puts indexes into folder, INDEX_references 
def salmon_index(genome):
    #os.system('salmon index -t '+str(genome)+' -i INDEX_'+str(genome))
    #print('salmon '+ genome)
    return 

#call grab_ref() and salmon_index() to build index for each reference genome
for i in grab_ref(ref_folder):
    salmon_index(i)

# Get Quants for Each Reference Genome on a Per Sample Basis
We want all of the runs from each sample to be fed into salmon at one time. This is currently robust to number of runs, but is not robust to library type or paired vs unpaired. If mulitple files, give -a and then a space seperated string of file names. Files must be of same library type and aligned to same transcriptome. 

loops through indexes and samples and runs quant for each sample and each index. 

fastqdump downloads data on the fly so we are not running prefetch

still need to test this robustly

In [42]:
index_names =[] #will use this later

for y in glob.glob('INDEX_'+ref_folder+'/*'):
    index_name = y.replace('INDEX_references/','').replace('.ffn.gz','')
    print('Index: '+index_name)
    index_names.append(index_name)
    #you could fairly easily change this to loop by experiment 
    #number or biosample number...need to talk about which one to choose
    for z in sra_run_table['Run'][3:5]:
        dump_directory = 'data/'+z
        #os.system('mkdir data/'+z)
        print('Dump Directory: '+dump_directory)
        #os.system('fastq-dump --outdir '+dump_directory+' -v --gzip --skip-technical --readids --split-3 --clip '+z)
        runs = grab_ref(dump_directory)
        output_name='quants/'+index_name+'/'+z+'_quant'
        print('Quant output:'+output_name)
        if len(runs)<1:
            print('Oops, no runs for '+z)
        if len(runs)==1:
            print('One run for '+j)
            print('Salmon input: '+runs[0])
            os.system('salmon quant -i '+y+' -l A '+runs[0]+' -p 8 --validateMappings -o '+output_name)
        if len(runs)>1:
            if "_1" in runs[0] or "_2" in runs[0]:
                print('Multiple paired runs for '+z)
                input_name='-1'
                for z in np.arange(0, len(runs)):
                    if "_1" in runs[z]:
                        input_name+= ' '+runs[z]
                input_name+=' -2'
                for z in np.arange(0, len(runs)):
                    if "_2" in runs[z]:
                        input_name+= ' '+runs[z]
            else:
                print('Multiple runs for '+z)
                input_name='-r'
                for z in np.arange(0, len(runs)):
                    input_name+= ' '+runs[z]
            print('Salmon Input: '+input_name)
            #os.system('salmon quant -i '+y+' -l A '+input_name+' -p 8 --validateMappings -o '+output_name)

Index: Pseudomonas_aeruginosa_UCBPP-PA14_109
Dump Directory: data/SRR8239646
Quant output:quants/Pseudomonas_aeruginosa_UCBPP-PA14_109/SRR8239646_quant
One run for data/SRR6483189
Salmon input: data/SRR8239646/SRR8239646.fastq.gz
Dump Directory: data/SRR6483189
Quant output:quants/Pseudomonas_aeruginosa_UCBPP-PA14_109/SRR6483189_quant
['data/SRR6483189/SRR6483189.fastq1.gz', 'data/SRR6483189/SRR6483189.fastq12.gz', 'data/SRR6483189/SRR6483189.fastq.gz']
Multiple runs for SRR6483189
Salmon Input: -r data/SRR6483189/SRR6483189.fastq1.gz data/SRR6483189/SRR6483189.fastq12.gz data/SRR6483189/SRR6483189.fastq.gz
Index: Pseudomonas_aeruginosa_PAO1_107
Dump Directory: data/SRR8239646
Quant output:quants/Pseudomonas_aeruginosa_PAO1_107/SRR8239646_quant
One run for data/SRR6483189
Salmon input: data/SRR8239646/SRR8239646.fastq.gz
Dump Directory: data/SRR6483189
Quant output:quants/Pseudomonas_aeruginosa_PAO1_107/SRR6483189_quant
['data/SRR6483189/SRR6483189.fastq1.gz', 'data/SRR6483189/SRR64831

# Storing Output

Copied Alex's code. 

Similar to running the quants, for each reference genome, we're going to grab the quants on a sample by sample basis and construct a data frame for each reference genome. The dataframes will be stored in a list. 

Not sure if row names are robust. I think they are, but I have not thought hard about it. 

In [9]:
#loop through indexes and build dataframe of output for each one
df_list = []
for i in index_names:
    glob_list = []
    replace = 'quants/'+i+'/'
    #print(i)
    for z in grab_ref('quants/'+i):
        #print(z)
        for file in glob.glob(z+'/*.sf'):
            glob_list.append(file)
            #print(file)
    expression_df = pd.DataFrame(
    pd.read_csv(file, sep="\t", index_col=0)["TPM"].
        rename(file.replace(replace, '').replace('_quant/quant.sf',''))
    for file in glob_list)
    df_list.append(expression_df)
    expression_df.to_csv(csv+'/aligned_to_'+i, sep='\t')
print(df_list)

[Name                    PGD1650835  PGD1650837  PGD1650839  PGD1650841  \
GD9-45172138_pstB_rep2   81.392938  138.626252   54.634600   60.881744   
GD1-45169164-WT_rep1    127.691467  219.547086   91.361946   91.925850   
GD3-45171150-pstB_rep1   91.208678  135.613848   63.026178   68.336412   
GD7-45169169_WT_rep2    116.833104  193.536140   73.984332   76.042705   

Name                    PGD1650843  PGD1650845  PGD1650847  PGD1650849  \
GD9-45172138_pstB_rep2   16.264421   20.309403  291.540767   16.550769   
GD1-45169164-WT_rep1     34.185101   38.716848  331.107661   27.048108   
GD3-45171150-pstB_rep1   19.362714   33.293163  239.210943   18.593997   
GD7-45169169_WT_rep2     28.673334   27.881871  260.109989   25.418221   

Name                    PGD1650851  PGD1650853  ...  PGD1662756  PGD1662758  \
GD9-45172138_pstB_rep2   24.736559   16.688346  ...   59.922889   70.892426   
GD1-45169164-WT_rep1     37.439678   28.778906  ...  112.710116  135.050741   
GD3-45171150-pstB_re